In [1]:
from connection import *
import dash
import psycopg2
from dash import Dash, html, dcc
import matplotlib.pyplot as plt
import numpy as np
import plotly.express as px
import pandas as pd
import seaborn as sns

Conexión exitosa
TABLA ALBUMES: album_id, nombre, fecha_lanzamiento
(1, '57', datetime.date(2024, 11, 8))
(2, 'Soltera (W Sound 01)', datetime.date(2024, 8, 6))
(3, 'MÃ\xadrame', datetime.date(2024, 4, 17))
(4, 'DEPORTIVO', datetime.date(2024, 11, 14))
(5, 'POLOS OPUESTOS REMIX (BLESSD VERSION)', datetime.date(2024, 10, 9))
(6, 'FERXXOCALIPSIS', datetime.date(2023, 12, 1))
(7, 'FDSR', datetime.date(2024, 10, 10))
(8, 'SE ME OLVIDA', datetime.date(2024, 8, 23))
(9, 'Cosas Pendientes', datetime.date(2024, 11, 21))
(10, 'Cosa Nuestra', datetime.date(2024, 11, 15))
(11, 'SQ (W Sound 04)', datetime.date(2024, 11, 20))
(12, 'Las Piponas (Remix)', datetime.date(2024, 10, 1))
(13, 'Si Antes Te Hubiera Conocido', datetime.date(2024, 6, 21))
(14, 'Duelo', datetime.date(2024, 11, 7))
(15, 'ImagÃ\xadnate', datetime.date(2024, 11, 7))
(16, 'UWAIE', datetime.date(2024, 8, 15))
(17, 'Ohnana', datetime.date(2024, 6, 6))
(18, 'Frente al Mar', datetime.date(2024, 9, 26))
(19, 'Tu Boda', datetime.date(20


# Gráfica / representación de la consulta 1

En esta sección, se presenta la gráfica de barras generada a partir de los datos obtenidos de la consulta 1. Esta consulta analiza cómo los atributos musicales de las canciones, como danceability, energy, valence y tempo, están relacionados con su éxito en los rankings (definido por su mejor posición alcanzada).

In [2]:
try:
    # Conexión a la base de datos
    connection = psycopg2.connect(
        host='localhost',
        user='postgres',  # Reemplaza con tu usuario de la base de datos
        password='2I@@`7ao*3}*',  # Reemplaza con tu contraseña
        database='proyecto'
    )
    print("Conexión exitosa")
    
    # Crear un cursor
    cursor = connection.cursor()
    
    # Ejecutar la consulta
    t_consulta1 = "SELECT c.spotify_id, c.nombre AS cancion, c.danceability, c.energy, c.valence, c.tempo, MIN(r.ranking_diario) AS mejor_ranking FROM Canciones c JOIN Rankings r ON c.spotify_id = r.spotify_id GROUP BY c.spotify_id, c.nombre, c.danceability, c.energy, c.valence, c.tempo HAVING MIN(r.ranking_diario) <= 10 ORDER BY mejor_ranking ASC;"

    cursor.execute(t_consulta1)
    rows = cursor.fetchall()
    
    # Obtener los nombres de las columnas
    column_names = [desc[0] for desc in cursor.description]

    # Crear un DataFrame
    df = pd.DataFrame(rows, columns=column_names)
    print(df)

    # Generar la gráfica de barras
    fig = px.bar(
        df,
        x='cancion',
        y='mejor_ranking',
        title='Mejor Ranking por Canción',
        labels={
            'mejor_ranking': 'Mejor Ranking',
            'cancion': 'Canción'
        }
    )

    # Mostrar la gráfica
    fig.show()
    
except Exception as error:
    print("Error al conectar o ejecutar la consulta:", error)
finally:
    if connection:
        cursor.close()
        connection.close()
        print("Conexión cerrada")

Conexión exitosa
                spotify_id                                cancion  \
0   0l0vcZMU7AOeQmUIREoI2U                       QuÃ© PasarÃ­a...   
1   3mg9jYNC1NihbthlglzrWP                                     57   
2   6hLsMjw00j3HGrkUjKwmgo                                   A200   
3   5MKbWaXeSQSmzmo7gPQha1                           Amor de Vago   
4   1YYfyJ2yCvGFCxwjsIqEK9                                  KhÃ©?   
5   03IpZ9yqJlo3EIgDQUgBSF                              Nueva Era   
6   1cOboCuWYI2osTOfolMRS6                                Tu Boda   
7   0c82YIGpQ5dCaXpTNCk4sb                               AlegrÃ­a   
8   4OLT65TRsOx3Iv1TlCcQVb                             Presidente   
9   6X6m4xmdFcz31p1h7Qg68c                             EL LOKERON   
10  6PhRhocBXP4y7NpciGmzra                       DespuÃ©s De La 1   
11  13BDiikG6y5o5cQTK0HpW6                   Soltera - W Sound 01   
12  1uFW6czrHk5PsK3s6gOI7C                                 Se Fue   
13  5AkLEnorFvb7W

Conexión cerrada


# Gráfica y representación de la consulta 2
En esta sección, se presenta la gráfica de barras a partir de los datos obtenidos de la consulta 2. Esta consulta encuentra la canción más exitosa de cada artista basándose en su mejor posición en los rankings.

In [8]:
try:
    # Conexión a la base de datos
    connection = psycopg2.connect(
        host='localhost',
        user='postgres',  # Reemplaza con tu usuario de la base de datos
        password='2I@@`7ao*3}*',  # Reemplaza con tu contraseña
        database='proyecto'
    )
    print("Conexión exitosa")
    
    # Crear un cursor
    cursor = connection.cursor()
    
    t_consulta2 = "SELECT a.nombre AS artista, c.nombre AS cancion, MIN(r.ranking_diario) AS mejor_posicion FROM Artistas a JOIN Cancion_Artista ca ON a.artista_id = ca.artista_id JOIN Canciones c ON ca.spotify_id = c.spotify_id JOIN Rankings r ON c.spotify_id = r.spotify_id GROUP BY a.nombre, c.nombre ORDER BY mejor_posicion ASC, a.nombre;"
    
    # Ejecutar la consulta
    cursor.execute(t_consulta2)
    rows = cursor.fetchall()

    # Crear un DataFrame
    df = pd.DataFrame(rows, columns=['artista', 'cancion', 'mejor_posicion'])

    # Generar la gráfica de barras
    fig = px.bar(
        df,
        x='artista',
        y='mejor_posicion',
        color='cancion',
        title='Mejor Posición de Canciones por Artista',
        labels={
            'mejor_posicion': 'Mejor Posición',
            'artista': 'Artista',
            'cancion': 'Canción'
        }
    )

    # Mostrar la gráfica
    fig.show()
except Exception as error:
    print("Error al conectar o ejecutar la consulta:", error)
finally:
    if connection:
        cursor.close()
        connection.close()
        print("Conexión cerrada")

Conexión exitosa


Conexión cerrada


# Gráfica y representación de la consulta 3
En esta sección, se presenta la gráfica de lineas a partir de los datos obtenidos de la consulta 3. Evalúa cómo varía la popularidad de las canciones a lo largo del tiempo observando su promedio de posición diaria (ranking_diario).

In [6]:
try:
    # Conexión a la base de datos
    connection = psycopg2.connect(
        host='localhost',
        user='postgres',  # Reemplaza con tu usuario de la base de datos
        password='2I@@`7ao*3}*',  # Reemplaza con tu contraseña
        database='proyecto'
    )
    print("Conexión exitosa")
    
    # Crear un cursor
    cursor = connection.cursor()
    
    t_consulta3 = "SELECT c.nombre AS cancion, s.fecha_snapshot, AVG(r.ranking_diario) AS promedio_ranking FROM Canciones c JOIN Rankings r ON c.spotify_id = r.spotify_id JOIN Snapshots s ON r.fecha_snapshot = s.fecha_snapshot GROUP BY c.nombre, s.fecha_snapshot ORDER BY c.nombre, s.fecha_snapshot"
    
    # Ejecutar la consulta
    cursor.execute(t_consulta3)
    rows = cursor.fetchall()

    # Crear un DataFrame
    df = pd.DataFrame(rows, columns=['cancion', 'fecha_snapshot', 'promedio_ranking'])

    # Convertir la columna de fecha a tipo datetime
    df['fecha_snapshot'] = pd.to_datetime(df['fecha_snapshot'])

    # Generar el gráfico de líneas
    fig = px.line(
        df,
        x='fecha_snapshot',
        y='promedio_ranking',
        color='cancion',
        title='Tendencias de Popularidad de Canciones a lo Largo del Tiempo',
        labels={
            'fecha_snapshot': 'Fecha',
            'promedio_ranking': 'Promedio de Ranking',
            'cancion': 'Canción'
        }
    )

    # Mostrar el gráfico
    fig.show()
except Exception as error:
    print("Error al conectar o ejecutar la consulta:", error)
finally:
    if connection:
        cursor.close()
        connection.close()
        print("Conexión cerrada")

Conexión exitosa


Conexión cerrada


# Gráfica y representación de la consulta 4
En esta sección, se presenta la gráfica de lineas a partir de los datos obtenidos de la consulta 4. Analiza la diversidad musical de cada país en términos de atributos como acousticness, instrumentalness, y danceability, junto con la cantidad de artistas que contribuyen a esa diversidad.

In [7]:
try:
    # Conexión a la base de datos
    connection = psycopg2.connect(
        host='localhost',
        user='postgres',  # Reemplaza con tu usuario de la base de datos
        password='2I@@`7ao*3}*',  # Reemplaza con tu contraseña
        database='proyecto'
    )
    print("Conexión exitosa")
    
    # Crear un cursor
    cursor = connection.cursor()
    
    t_consulta4 = "SELECT p.nombre AS pais, COUNT(DISTINCT a.artista_id) AS cantidad_artistas, AVG(c.acousticness) AS promedio_acousticness, AVG(c.instrumentalness) AS promedio_instrumentalness, STDDEV(c.danceability) AS desviacion_danceability, STDDEV(c.energy) AS desviacion_energy FROM Rankings r JOIN Canciones c ON r.spotify_id = c.spotify_id JOIN Paises p ON r.pais_codigo = p.pais_codigo JOIN Cancion_Artista ca ON c.spotify_id = ca.spotify_id JOIN Artistas a ON ca.artista_id = a.artista_id GROUP BY p.nombre ORDER BY cantidad_artistas DESC, p.nombre;"

    # Ejecutar la consulta
    cursor.execute(t_consulta4)
    rows = cursor.fetchall()

    # Crear un DataFrame
    df = pd.DataFrame(rows, columns=['pais', 'cantidad_artistas', 'promedio_acousticness', 'promedio_instrumentalness', 'desviacion_danceability', 'desviacion_energy'])

    # Generar el gráfico de barras
    fig = px.bar(
        df,
        x='pais',
        y='cantidad_artistas',
        title='Número de Artistas por País',
        labels={
            'cantidad_artistas': 'Cantidad de Artistas',
            'pais': 'País'
        }
    )

    # Mostrar el gráfico
    fig.show()
except Exception as error:
    print("Error al conectar o ejecutar la consulta:", error)
finally:
    if connection:
        cursor.close()
        connection.close()
        print("Conexión cerrada")

Conexión exitosa


Conexión cerrada
